In [12]:
# -----------------All IMPORTS--------------------------------------------------------------
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor

#----------------------------time seriesimports------------
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from pandas import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from math import sqrt
import re
import itertools

In [13]:
#---------------- DATA File READ and DATA MUNJING
data=pd.read_csv("D:\\YashuProjects\\Regression_anal\\Reg_V1\\company\\Summed_data1.0.csv")
time_slices = ['12:00 AM','12:30 AM','01:00 AM','01:30 AM','02:00 AM','02:30 AM','03:00 AM','03:30 AM','04:00 AM','04:30 AM','05:00 AM',
               '05:30 AM','06:00 AM','06:30 AM','07:00 AM','07:30 AM','08:00 AM','08:30 AM','09:00 AM','09:30 AM','10:00 AM','10:30 AM',
               '11:00 AM','11:30 AM','12:00 PM','12:30 PM','01:00 PM','01:30 PM','02:00 PM','02:30 PM','03:00 PM','03:30 PM','04:00 PM',
               '04:30 PM','05:00 PM','05:30 PM','06:00 PM','06:30 PM','07:00 PM','07:30 PM','08:00 PM','08:30 PM','09:00 PM','09:30 PM',
               '10:00 PM','10:30 PM','11:00 PM','11:30 PM']

Staff_dictionary={'Billing':1, 'CHSI':2, 'New Products':3, 'Order Entry':4,
                  'Repair':5, 'Sales':6}

In [14]:
#----------------------------------  Time Series Function call -----------------------------------------

In [15]:
# -------------------------------------------ARIMA MODEL(TRAIN AND PREDICT) ------------------------------------
def arima(df, no_of_iterations):
    X = df['ACT Calls'].values.tolist()
    train, test = X[:int(len(X)*0.8)], X[int(len(X)*0.8):]
    actual_value = test
    history=train
    predictions = list()
    original = list()
    fcst_list = df['FCST Calls'].values.tolist()[int(len(X)*0.8):]
    for t in range(no_of_iterations):
        try:
            print('Iteration: ', t+1, '/', no_of_iterations)
            model = ARIMA(history, order=(4,1,3))
            model_fit = model.fit(disp=0)
            
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            original.append(test[t])
            history.append(test[t])
            
            print('predicted = %f, expected = %f, forecasted = %f' % (yhat, test[t], fcst_list[t]))
        except Exception as e:
            print(e.msg)
    
    summation = error_rate = 0    
    for i in range(no_of_iterations):
        summation += predictions[i] - test[i]

    for i in range(no_of_iterations):
        error_rate += (predictions[i] - test[i])/predictions[i]

    summation1 = error_rate1 = 0    
    for i in range(no_of_iterations):
        summation1 += fcst_list[i] - test[i]

    for i in range(no_of_iterations):
        error_rate1 += (fcst_list[i] - test[i])/fcst_list[i]
    print()
    print("Average error - ","\nFor our predictions: ", summation/no_of_iterations, " For given forcasted values: ", summation1/no_of_iterations)
    print("Average error rate - ", "\nFor our predictions: ", error_rate/no_of_iterations,  " For given forcasted values: ", error_rate1/no_of_iterations)
    
    print("MAE: ", "\nFor our predictions: ", mean_absolute_error(original, predictions), " For given forcasted values: ", mean_absolute_error(original, fcst_list[:no_of_iterations]))
    print("R2 Score: ",  "\nFor our predictions: ", r2_score(original, predictions), " For given forcasted values: ", r2_score(original, fcst_list[:no_of_iterations]))

In [16]:
#-----------Invoking Function for ARIMA Model 
arima(data,5)

Iteration:  1 / 5
predicted = 139.405527, expected = 172.000000, forecasted = 189.000000
Iteration:  2 / 5
predicted = 248.945715, expected = 188.000000, forecasted = 220.000000
Iteration:  3 / 5
predicted = 164.871021, expected = 197.000000, forecasted = 212.000000
Iteration:  4 / 5
predicted = 184.493289, expected = 217.000000, forecasted = 209.000000
Iteration:  5 / 5
predicted = 210.686615, expected = 189.000000, forecasted = 241.000000

Average error -  
For our predictions:  [-2.91956675]  For given forcasted values:  21.6
Average error rate -  
For our predictions:  [-0.05142602]  For given forcasted values:  0.09672929505516335
MAE:  
For our predictions:  35.97249851109815  For given forcasted values:  24.8
R2 Score:  
For our predictions:  -5.835675193623116  For given forcasted values:  -3.012299664554603


In [17]:
#-----------------------------------------REGRESSION DATA PREPARING START -------------------------------------------------------

In [18]:
# ONE TIME FUNCTION FOR PREPARING DATA
def encode_time(forcast_time):
    quarter_slice = next((s for s in time_slices if datetime.strptime(str(forcast_time),"%I:%M %p")<datetime.strptime(str(s),"%I:%M %p")),'03:00 AM')
    return time_slices.index(quarter_slice)+1

def encode_weekday(forcast_date):
    return (datetime.strptime(forcast_date,"%m/%d/%Y").weekday() + 1)

def encode_weekno(forcast_date):
    return datetime.strptime(forcast_date,"%m/%d/%Y").strftime("%W")


# cols=['Staff Group','ACT Calls','WeekDay', 'Interval']
cols=['Staff Group','ACT Calls','WeekDay', 'National_holdiday', 'Epl_days', 'Interval',"weekno"]
data['Staff Group']=data['Staff Group'].map(lambda x:" ".join(str(x).split()[1:]))
data['Staff Group']=data['Staff Group'].map(Staff_dictionary)
data['Interval'] = data['Interval'].map(encode_time)
data['WeekDay'] = data['Date'].map(encode_weekday)
data['weekno'] = data['Date'].map(encode_weekno)

In [19]:
#---------------------------------TRAIN TEST SPLIT-----------------------------------
Y = data['ACT Calls']
data = data[cols]
data.drop("ACT Calls",axis=1,inplace=True)
X_train, X_test, Y_train, Y_test = train_test_split(data, Y, test_size = 0.2, random_state=42)

In [20]:
def getPredictAnalysis(df):
    #     print(df)
    summation = 0
    error_rate = 0
    for i,row in df.iterrows():
        summation += (row['predicted'] - row['Actual'])
        if row['predicted']!=0:
            error_rate += (row['predicted'] - row['Actual'])/row['predicted']

    print("Average error : ",summation/len(df))
    print("Error rate : ",error_rate/len(df))

def getForecastAnalysis(df):
    summation = 0
    error_rate = 0
    for i,row in df.iterrows():
        summation += (row['FCST Calls'] - row['Actual'])
        if row['FCST Calls']!=0:
            error_rate += (row['FCST Calls'] - row['Actual'])/row['FCST Calls']

    print("Average error : ",summation/len(df))
    print("Error rate : ",error_rate/len(df))

def getAnalysis(df,column_name):
    possible_values=[0,1]
    print("------*****************************-------")
    print(column_name)
    #     print(df[column_name])
    for v in possible_values:
        print("Predicted value corrections")
        #         print(column_name,v)
        getPredictAnalysis(df[df[column_name]==v])
        print("Original forecast values corrections")
#         getForecastAnalysis(df[df[column_name]==v])

In [21]:
def getResults(df):
    df['predicted']=df['predicted'].astype(float)
    df['Actual']=df['Actual'].astype(float)
    #     df['FCST Calls']=df['FCST Calls'].astype(float)
    df.to_csv("Prepared_dataset.csv",index=False)
    groupedData = df.groupby("Staff Group")
    results_cols = ['National_holdiday','Epl_days','nfl_days']
    print("Predicted Value corrections")
    getPredictAnalysis(df)
    # print("Original Forecast value Corrections")
    # getForecastAnalysis(df)
    # for name,group in groupedData:
    #     Staff_group = next((k for k,v in Staff_dictionary.items() if v == name), None)
    #     print("----------+++++++++++++++++++++++++++-------------------")
    #     print("For Staff Group : {}".format(Staff_group))
    #     for col in results_cols:
    #         getAnalysis(groupedData.get_group(name),col)

def linearmodel(X_train, X_test, Y_train, Y_test):
    lin_model = LinearRegression()
    lin_model.fit(X_train, Y_train)
    y_predict = lin_model.predict(X_test)
    print('MAE is {}'.format(mean_absolute_error(Y_test, y_predict)))
    print('R2 score is {}'.format(r2_score(Y_test, y_predict)))
    X_test['predicted'] = y_predict
    X_test['Actual'] = Y_test
    #     X_test.to_csv("Linear Regression.csv",index=False)
    getResults(X_test)

# Function for Random Forrest Regressor 
def randomModel(X_train, X_test, Y_train, Y_test):
    regressor = RandomForestRegressor(n_estimators=700, random_state=42,max_features="sqrt")
    regressor.fit(X_train, Y_train)
    y_pred = regressor.predict(X_test)
    print('R2 score is :',r2_score(Y_test, y_pred))
    print("Training set score: {:.2f}".format(regressor.score(X_train, Y_train)))
    print("Test set score: {:.2f}".format(regressor.score(X_test, Y_test)))
    print('Mean Absolute Error:', mean_absolute_error(Y_test, y_pred))
    X_test['predicted'] = y_pred
    X_test['Actual'] = Y_test

    X_test.to_csv("Final_Prediction_results.csv",index=False)
    getResults(X_test)

In [22]:
# Caliing Random Forrest Regresor 
randomModel(X_train, X_test, Y_train, Y_test)

R2 score is : 0.9491363995365396
Training set score: 0.98
Test set score: 0.95
Mean Absolute Error: 23.463223960531113


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexin

Predicted Value corrections
Average error :  -0.002769388095764265
Error rate :  -0.04650212975607963
